In [ ]:
%run "../01-check_setup.ipynb"

In [ ]:
hana_table_name="FAQ"

In [ ]:
# from hana_ml.dataframe import ConnectionContext

## Create a dataframe for a HANA table
hdf_faq_bronze = myconn.table(hana_table_name)

In [ ]:
hdf_faq_bronze.head().collect()

## Generating Text Embeddings in SAP HANA Cloud

In [ ]:
content_column = 'content'

In [ ]:
print(f"""Number of records selected for further processing: {hdf_faq_bronze.count()}""")

In [ ]:
### Generating Text Embeddings in SAP HANA Cloud with the new PAL function, function available with hana-ml 2.23.
from hana_ml.text.pal_embeddings import PALEmbeddings
pe = PALEmbeddings(model_version="SAP_GXY.20250407")
hdf_faq_silver = pe.fit_transform(hdf_faq_bronze.add_id(), key="ID", target=[f"{content_column}"], thread_number=10, batch_size=10) #, max_token_num=512
print(f"{hdf_faq_silver.count()} records processed in {round(pe.runtime, 3)} sec")

In [ ]:
hdf_faq_silver.get_table_structure()

In [ ]:
hdf_faq_silver.head(1).collect()

In [ ]:
hdf_faq_silver.select_statement

In [ ]:
hdf_faq_silver=hdf_faq_silver.save(where="#FAQ_EMBEDDINGS", force=True)

## Semantic search in FAQ

In [ ]:
prompt="What are origins of the Python name?"

In [ ]:
df_result = myconn.sql(
    f"""SELECT
--    COSINE_SIMILARITY(VECTOR_EMBEDDING('{prompt}', 'DOCUMENT', 'SAP_NEB.20240715'), "VECTOR_COL_{content_column}") AS "SIMILARITY",
    COSINE_SIMILARITY(VECTOR_EMBEDDING('{prompt}', 'DOCUMENT', 'SAP_GXY.20250407'), "VECTOR_COL_{content_column}") AS "SIMILARITY",
    "ID", "{content_column}"
    FROM ({hdf_faq_silver.select_statement})
    ORDER BY 1 DESC;
    """
).collect()

In [ ]:
df_result.head(3)

In [ ]:
# Print the rows of the 'content' column
print(df_result['content'][0])

In [ ]:
from IPython.display import Markdown

# Convert the rows of the 'content' column to markdown format
display(Markdown(df_result['content'][0]))